# Modifying existing TopoJSON file with Colombia's towns and departments

This work started due to an interest for understanding the origin of a shape map in a Power BI dashboard. Unfortunately, I wasn't able to track down the creator of the Power BI dashboard that contained a shape map of Colombia's departments.

I started a web search and found a TopoJSON file with Colombia's towns and departments in [John Guerra's Observable profile](https://observablehq.com/@john-guerra/topojson-colombia). I found another TopoJSON file with Colombia's towns and departments in [Javier Moreno' GitHub profile](https://github.com/finiterank/mapa-colombia-js). I decided to start my exploration with the former since it's a more recent file.

Initially, my exploration was aimed at understanding what kind of information was contained in the TopoJSON file so that I could extract the information linked only to departments. I also used notepad++ to help me explore and understand the structure of the file. This notebook contains this exploratory work.

Extracting the departments' info works. I use the created file `mapa_departamentos.json` in a Power BI dashboard and can see a map of Colombia with its departments.

![dashboard](../images/dashboard_departments.png)

Note San Andrés, Providencia and Santa Catalina (archipelago hereafter) are appropriately shown in scale and location. However, viz-wise, this could be improved. The Power BI dashboard that originated this work had the archipelago closer to mainland and larger. However, I didn't want to rely on replicating this Power BI dashboard file (plus the necessary edits to the dashboard and its source data to fit its new purpose) everytime I wanted a shape map of Colombia's departments. This led to a further exploration documented in the notebook `new_topojson_map/create_map.ipynb`.

## 1. Loading file

In [1]:
import json

json_file = 'Colombia_departamentos_municipios_poblacion-topov2.json'
with open(json_file, 'r') as file:
    map_json = json.load(file)

## 2. Data exploration

## objects

The key $objects$ has a **dictionary** with the following two keys:

- $MGN\_ANM\_MPIOS$
- $MGN\_ANM\_DPTOS$

Each key has a **dictionary** with the corresponding geometries. Let's focus on $MGN\_ANM\_DPTOS$, which corresponds to the deparments' key.

### MGN_ANM_DPTOS

The key $MGN\_ANM\_DPTOS$ has a **dictionary** with the following two keys:

- $type$
- $geometries$

#### geometries

The key $geometries$ has a **list** of *dictionaries*. Each dictionary contains the information of a department in the following three keys:

- $arcs$
- $type$
- $properties$

##### properties

The key $properties$ has dictionary, where the code of a deparment is stored under the key $DPTO\_CCDGO$. Let's explore this structure below.

In [2]:
map_json['objects']['MGN_ANM_DPTOS']['geometries'][0]['properties']

{'DPTO_CCDGO': '18',
 'DPTO_CNMBR': 'CAQUETÁ',
 'VERSION': 2018,
 'AREA': 90103008160.2,
 'LATITUD': 0.79855619489,
 'LONGITUD': -73.9594675611,
 'TSP16_HOG': 116166,
 'STP27_PERS': 359602}

In [3]:
map_json['objects']['MGN_ANM_DPTOS']['geometries'][0]['properties']['DPTO_CCDGO']

'18'

In [4]:
map_json['objects']['MGN_ANM_DPTOS']['geometries'][1]['properties']

{'DPTO_CCDGO': '19',
 'DPTO_CNMBR': 'CAUCA',
 'VERSION': 2018,
 'AREA': 31242914793,
 'LATITUD': 2.39683388683,
 'LONGITUD': -76.8242328283,
 'TSP16_HOG': 432493,
 'STP27_PERS': 1243503}

Let's loop...

In [5]:
for department in map_json['objects']['MGN_ANM_DPTOS']['geometries']:
    print(department['properties'])

{'DPTO_CCDGO': '18', 'DPTO_CNMBR': 'CAQUETÁ', 'VERSION': 2018, 'AREA': 90103008160.2, 'LATITUD': 0.79855619489, 'LONGITUD': -73.9594675611, 'TSP16_HOG': 116166, 'STP27_PERS': 359602}
{'DPTO_CCDGO': '19', 'DPTO_CNMBR': 'CAUCA', 'VERSION': 2018, 'AREA': 31242914793, 'LATITUD': 2.39683388683, 'LONGITUD': -76.8242328283, 'TSP16_HOG': 432493, 'STP27_PERS': 1243503}
{'DPTO_CCDGO': '86', 'DPTO_CNMBR': 'PUTUMAYO', 'VERSION': 2018, 'AREA': 25976283135, 'LATITUD': 0.45225996697, 'LONGITUD': -75.8559119498, 'TSP16_HOG': 107053, 'STP27_PERS': 283197}
{'DPTO_CCDGO': '76', 'DPTO_CNMBR': 'VALLE DEL CAUCA', 'VERSION': 2018, 'AREA': 20665544524.7, 'LATITUD': 3.85885827935, 'LONGITUD': -76.5186942422, 'TSP16_HOG': 1267039, 'STP27_PERS': 3789874}
{'DPTO_CCDGO': '94', 'DPTO_CNMBR': 'GUAINÍA', 'VERSION': 2018, 'AREA': 71289354481.1, 'LATITUD': 2.72784286544, 'LONGITUD': -68.8166127213, 'TSP16_HOG': 9953, 'STP27_PERS': 44431}
{'DPTO_CCDGO': '99', 'DPTO_CNMBR': 'VICHADA', 'VERSION': 2018, 'AREA': 10006337059

## 3. Departaments

We define a function to remove the town's key from the JSON structure or dictionary.

In [6]:
def delete_nested_key(data, parent_key, key_to_delete):
    """
    This function deletes a nested key from a dictionary.

    Args:
        data: JSON structure
        parent_key: parent key
        key_to_delete: nested key

    Returns:
        Dictionary without the towns' key and its value     
    """
    
    if parent_key in data and key_to_delete in data[parent_key]:
        del data[parent_key][key_to_delete]

    return data

In [7]:
parent_key = 'objects'
key_to_delete = 'MGN_ANM_MPIOS'
map_json = delete_nested_key(map_json, parent_key, key_to_delete)

We change the type of the department codes from $str$ to $int$ so that Power BI can relate them to the data (When loading the DIVIPOLA codes into Power BI, Power BI automatically casts them into numbers).

In [8]:
for department in map_json['objects']['MGN_ANM_DPTOS']['geometries']:
    department['properties']['DPTO_CCDGO'] = int(department['properties']['DPTO_CCDGO'])

We save the data.

In [9]:
output_file = 'mapa_departamentos.json'
with open(output_file, 'w') as file:
    json.dump(map_json, file)

## 4. Towns

We can do something similar for towns:

```python
with open(json_file, 'r') as file:
    map_json = json.load(file)

# We delete the departments
parent_key = 'objects'
key_to_delete = 'MGN_ANM_DPTOS'
map_json = delete_nested_key(map_json, parent_key, key_to_delete)

# We cast the towns' codes
for town in map_json['objects']['MGN_ANM_MPIOS']['geometries']:
    town['properties']['MPIO_CDPMP'] = int(town['properties']['MPIO_CDPMP'])

# We save the data
output_file = 'mapa_municipios.json'
with open(output_file, 'w') as file:
    json.dump(map_json, file)
```

## Additional sources

Some additional sources I checked were:

- [John Guerra's forum on GeoJSON map of Colombia](https://gist.github.com/john-guerra/43c7656821069d00dcbc)
- [Ayuda - Creación de mapa de municipios de Colombia](https://community.fabric.microsoft.com/t5/Translated-Spanish-Desktop/Ayuda-Creaci%c3%b3n-de-mapa-municipios-Colombia/td-p/2901751)
- [Creating custom shape map json file](https://community.fabric.microsoft.com/t5/Desktop/Creating-custom-shape-map-json-file/td-p/1910576)
- [Create Shape Map visualizations in Power BI Desktop (preview)](https://learn.microsoft.com/en-us/power-bi/visuals/desktop-shape-map)